In [21]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import torch
import matplotlib.pyplot as plt
from joblib import dump
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [24]:
X = pd.read_csv("../../processed_data.csv")
print(len(X[X.click_bool == 1.0]))
#X_train = X.head(2000000)
X_train = X.head(2000000)
X_val = X.tail(397656)
#X_val = X.tail(50000)
len(X_val)

106079


397656

In [25]:
X_train.head()

,prop_starrating,prop_location_score1,price_usd,promotion_flag,srch_length_of_stay,srch_adults_count,srch_children_count,srch_saturday_night_bool,click_bool,booking_bool,price_rating_ratio
0,3.0,2.83,104.77,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.028634
1,4.0,2.20,170.74,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.023427
2,3.0,2.20,179.80,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.016685
3,2.0,2.83,602.77,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.003318
4,4.0,2.64,143.58,0.0,1.0,4.0,0.0,1.0,0.0,0.0,0.027859


In [3]:
#Validation set

# Remove the label from the dataset and create Y vector
val = X_val.to_numpy()#[:100000,:]
print(val.shape)
# 8: click_bool, 9: booking_bool
exclude = [8,9, 10]
X_val = np.delete(val, exclude, axis=1)


(50000, 11)


In [4]:
Y_val = val[:,[8,9]]
Y_val_rel = np.zeros(Y_val.shape[0])
Y_val_rel[np.argwhere(Y_val[:,0]==1)[:,0]] = 2
Y_val_rel[np.argwhere(Y_val[:,1]==1)[:,0]] = 5
Y_val_rel[:100]

array([0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [26]:
len(X_train[X_train.click_bool == 1.0])

88464

In [27]:
X_zero = X_train[X_train.click_bool == 0.0].sample(100000, random_state=1)
X_one = X_train.loc[X_train.click_bool == 1.0]
train = pd.concat([X_zero, X_one], ignore_index=True, sort=False)
len(train)

188464

In [28]:
# Remove the label from the dataset and create Y vector
train = train.to_numpy()#[:100000,:]
np.save("Train_full_balanced", train)
print(train.shape)
# 8: click_bool, 9: booking_bool
exclude = [8,9]
Y_train = train[:,8]
print(Y_train[:20])
X_train = np.delete(train, exclude, axis=1)
print(X_train.shape, Y_train.shape)

(188464, 11)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(188464, 9) (188464,)


In [29]:
(Y_train==0).sum()

100000

In [9]:
scaler = StandardScaler().fit(X_train)
dump(scaler, "scaler_svm_train.joblib")
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
print(X_train.shape)

(47069, 8)


In [ ]:
#Model = svm.SVC(decision_function_shape="ovr")
Model = svm.SVC(decision_function_shape="ovr", kernel="linear", verbose=True, probability=True)
Model.fit(X_train,Y_train)

#predictions = Model.predict(scaler.transform(X_val))
#print("Accuracy: ", np.mean(predictions==Y_val))
dump(Model, "svm.joblib_subset")

In [ ]:
"""
from sklearn.ensemble import RandomForestClassifier

Model = RandomForestClassifier(max_depth=2, random_state=0, class_weight="balanced")
Model.fit(X_train,Y_train)

predictions = Model.predict(scaler.transform(X_val))
print("Accuracy: ", np.mean(predictions==Y_val))


In [10]:
from sklearn.metrics import ndcg_score

In [ ]:
predictions = Model.predict_proba(scaler.transform(X_val))

In [ ]:
#print(Y_val_rel.shape)
#print(predictions[:,1].shape)

print(predictions[:,1].shape)

Y_val_rel = np.array([Y_val_rel])
prediction_rel = np.array([predictions[:,1]])
print(Y_val_rel.shape)
print(prediction_rel.shape)

In [ ]:
ndcg_score(Y_val_rel, prediction_rel)

In [11]:
import xgboost as xgb

In [18]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

xg_reg.fit(X_train,Y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [19]:
preds = xg_reg.predict(scaler.transform(X_val))
print(preds[:50])

[0.4541862  0.45697123 0.47066492 0.47066492 0.4364699  0.45697123
 0.4364699  0.4364699  0.4541862  0.4364699  0.4541862  0.4364699
 0.45697123 0.4364699  0.45697123 0.4364699  0.4678799  0.4501636
 0.4364699  0.45697123 0.4364699  0.45697123 0.4678799  0.47066492
 0.4501636  0.4501636  0.4501636  0.4364699  0.4364699  0.4501636
 0.45697123 0.47066492 0.45697123 0.4364699  0.4364699  0.45697123
 0.4541862  0.4364699  0.4678799  0.45697123 0.45697123 0.4541862
 0.46565947 0.46565947 0.46565947 0.46565947 0.46565947 0.4422112
 0.4585135  0.4585135 ]


In [20]:
ndcg_score(np.array([Y_val_rel]), np.array([preds]))

0.6495846503033397